In [1]:
!nvidia-smi

Sun Jun  8 03:40:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install required Python libraries
!pip install ultralytics roboflow pytesseract opencv-python-headless easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB

In [3]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.8/112.6 GB disk)


In [8]:
from ultralytics import YOLO
import os
import cv2
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
from roboflow import Roboflow
import easyocr
from IPython.display import Image, display

# Hàm hiển thị ảnh trong Colab
def show_image(image, title="Image"):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 6))
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [4]:
def download_datasets():
    try:
        from roboflow import Roboflow
        rf = Roboflow(api_key="b1CmorT7gd3GwVuBjzKl")  # Thay bằng khóa API Roboflow của bạn

        # Tải tập dữ liệu Vehicle Registration Plates
        project2 = rf.workspace("augmented-startups").project("vehicle-registration-plates-trudk")
        dataset2 = project2.version(1).download("yolov8")
        print(f"Đã tải Vehicle Registration Plates Dataset vào {dataset2.location}")

        # Tải tập dữ liệu License Plate Number
        project3 = rf.workspace("project-enduni").project("license-plate-number-uqwhx")
        dataset3 = project3.version(2).download("yolov8")
        print(f"Đã tải License Plate Number Dataset vào {dataset3.location}")

        return dataset2.location, dataset3.location
    except Exception as e:
        print(f"Lỗi khi tải tập dữ liệu: {e}")
        return None, None

dataset2_path, dataset3_path = download_datasets()

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Vehicle-Registration-Plates-1 in yolov8:: 100%|██████████| 17658/17658 [00:02<00:00, 8344.95it/s]

Đã tải Vehicle Registration Plates Dataset vào /content/Vehicle-Registration-Plates-1
loading Roboflow workspace...


loading Roboflow project...



Extracting Dataset Version Zip to License-Plate-Number-2 in yolov8:: 100%|██████████| 12400/12400 [00:01<00:00, 11190.93it/s]

Đã tải License Plate Number Dataset vào /content/License-Plate-Number-2


In [5]:
def train_yolo_model(data_path):
    try:
        from ultralytics import YOLO
        model = YOLO("yolov8n.pt")  # Tải mô hình YOLOv8n

        # Huấn luyện mô hình
        results = model.train(
            data=f"{data_path}/data.yaml",
            epochs=20,
            imgsz=640,
            batch=16,
            name='license_plate_detection',
            device=0  # Sử dụng GPU
        )
        print("Huấn luyện hoàn tất!")
        return model, results
    except Exception as e:
        print(f"Lỗi khi huấn luyện mô hình: {e}")
        return None, None

if dataset2_path:
    plate_detection_model, train_results = train_yolo_model(dataset2_path)
else:
    print("Không thể huấn luyện do lỗi tải tập dữ liệu")

100%|██████████| 6.25M/6.25M [00:00<00:00, 98.2MB/s]


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Vehicle-Registration-Plates-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license_plate_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0

100%|██████████| 755k/755k [00:00<00:00, 22.9MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

      20/20      2.97G      1.014     0.3911      1.021         17        640: 100%|██████████| 386/386 [01:35<00:00,  4.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:11<00:00,  4.85it/s]

                   all       1765       1840      0.987      0.953      0.986      0.719



20 epochs completed in 0.617 hours.
Optimizer stripped from runs/detect/license_plate_detection/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/license_plate_detection/weights/best.pt, 6.2MB

Validating runs/detect/license_plate_detection/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:13<00:00,  4.19it/s]


                   all       1765       1840      0.982      0.958      0.986       0.72
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/license_plate_detection
Huấn luyện hoàn tất!


In [6]:
# Đánh giá mô hình phát hiện biển số
def evaluate_model(model, data_path):
    try:
        # Đánh giá trên tập validation
        val_results = model.val(data=f"{data_path}/data.yaml", split="val")
        print("Kết quả validation:")
        print(f"mAP50: {val_results.box.map50:.3f}, mAP50-95: {val_results.box.map:.3f}")

        # Đánh giá trên tập test
        test_results = model.val(data=f"{data_path}/data.yaml", split="test")
        print("Kết quả test:")
        print(f"mAP50: {test_results.box.map50:.3f}, mAP50-95: {test_results.box.map:.3f}")

        return val_results, test_results
    except Exception as e:
        print(f"Lỗi khi đánh giá mô hình: {e}")
        return None, None

# Chạy đánh giá
if plate_detection_model:
    val_results, test_results = evaluate_model(plate_detection_model, dataset2_path)
else:
    print("Không thể đánh giá do lỗi huấn luyện mô hình")

Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1244.5±1014.4 MB/s, size: 44.4 KB)


val: Scanning /content/Vehicle-Registration-Plates-1/valid/labels.cache... 1765 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1765/1765 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:14<00:00,  7.92it/s]


                   all       1765       1840      0.983      0.958      0.986      0.719
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/license_plate_detection2
Kết quả validation:
mAP50: 0.986, mAP50-95: 0.719
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 784.1±231.6 MB/s, size: 20.4 KB)


val: Scanning /content/Vehicle-Registration-Plates-1/test/labels... 882 images, 0 backgrounds, 0 corrupt: 100%|██████████| 882/882 [00:00<00:00, 2496.62it/s]

val: New cache created: /content/Vehicle-Registration-Plates-1/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:07<00:00,  7.87it/s]


                   all        882        902      0.993      0.969      0.992      0.727
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/license_plate_detection3
Kết quả test:
mAP50: 0.992, mAP50-95: 0.727


In [ ]:
# Huấn luyện mô hình nhận dạng ký tự với YOLOv8n
def train_character_recognition_model(data_path):
    try:
        model = YOLO("yolov8n.pt")  # Tải mô hình YOLOv8n mới

        # Huấn luyện mô hình cho nhận dạng ký tự
        results = model.train(
            data=f"{data_path}/data.yaml",
            epochs=50,  # Tăng epochs cho nhận dạng ký tự
            imgsz=640,
            batch=16,
            name='character_recognition',
            device=0  # Sử dụng GPU
        )
        print("Huấn luyện mô hình nhận dạng ký tự hoàn tất!")
        return model, results
    except Exception as e:
        print(f"Lỗi khi huấn luyện mô hình nhận dạng ký tự: {e}")
        return None, None

# Chạy huấn luyện cho nhận dạng ký tự
if dataset3_path:
    character_model, char_train_results = train_character_recognition_model(dataset3_path)
else:
    print("Không thể huấn luyện mô hình nhận dạng ký tự do lỗi tải dataset")

Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/License-Plate-Number-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=character_recognition, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

train: Scanning /content/License-Plate-Number-2/train/labels... 4750 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4750/4750 [00:01<00:00, 2377.66it/s]


train: New cache created: /content/License-Plate-Number-2/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.2±0.5 ms, read: 86.1±42.5 MB/s, size: 3.4 KB)


val: Scanning /content/License-Plate-Number-2/valid/labels... 707 images, 0 backgrounds, 0 corrupt: 100%|██████████| 707/707 [00:00<00:00, 928.69it/s]

val: New cache created: /content/License-Plate-Number-2/valid/labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      45/50      3.11G     0.8835     0.3744      1.118        107        640: 100%|██████████| 297/297 [01:10<00:00,  4.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:06<00:00,  3.81it/s]

                   all        707       5736      0.968      0.959      0.976      0.732



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      46/50      3.13G     0.8765     0.3714       1.11        123        640:  27%|██▋       | 80/297 [00:18<00:50,  4.28it/s]

In [ ]:
# Huấn luyện mô hình nhận dạng ký tự với YOLOv8s
def train_character_recognition_model_v8s(data_path):
    try:
        model = YOLO("yolov8s.pt")  # Tải mô hình YOLOv8s (lớn hơn v8n)

        # Huấn luyện mô hình YOLOv8s cho nhận dạng ký tự
        results = model.train(
            data=f"{data_path}/data.yaml",
            epochs=50,  # Tăng epochs cho nhận dạng ký tự
            imgsz=640,
            batch=16,
            name='character_recognition_v8s',  # Tên khác để phân biệt
            device=0  # Sử dụng GPU
        )

        print("Huấn luyện mô hình YOLOv8s nhận dạng ký tự hoàn tất!")
        return model, results

    except Exception as e:
        print(f"Lỗi khi huấn luyện mô hình YOLOv8s nhận dạng ký tự: {e}")
        return None, None

# Chạy huấn luyện cho nhận dạng ký tự với YOLOv8s
if dataset3_path:
    character_model_v8s, char_train_results_v8s = train_character_recognition_model_v8s(dataset3_path)
else:
    print("Không thể huấn luyện mô hình YOLOv8s nhận dạng ký tự do lỗi tải dataset")

In [ ]:
# Đánh giá mô hình nhận dạng ký tự (YOLOv8n)
if character_model:
    print("Đánh giá mô hình nhận dạng ký tự:")
    char_val_results, char_test_results = evaluate_model(character_model, dataset3_path)
else:
    print("Không thể đánh giá mô hình nhận dạng ký tự")

# Đánh giá mô hình nhận dạng ký tự (YOLOv8s)
if character_model_v8s:
    print("Đánh giá mô hình YOLOv8s nhận dạng ký tự:")
    char_val_results_v8s, char_test_results_v8s = evaluate_model(character_model_v8s, dataset3_path)
else:
    print("Không thể đánh giá mô hình YOLOv8s nhận dạng ký tự")

In [ ]:
def compare_models_performance():
    print("\n" + "="*50)
    print("SO SÁNH KẾT QUẢ GIỮA YOLOv8n VÀ YOLOv8s")
    print("="*50)

    if character_model and character_model_v8s:
        print("\n--- YOLOv8n Character Recognition ---")
        if char_val_results:
            try:
                # Truy cập thuộc tính box.map50 và box.map
                map50 = getattr(char_val_results.box, 'map50', 'N/A')
                map = getattr(char_val_results.box, 'map', 'N/A')
                print(f"Validation mAP50: {map50}")
                print(f"Validation mAP50-95: {map}")
            except:
                print("Không thể truy cập metrics của YOLOv8n")

        if char_test_results:
            try:
                map50 = getattr(char_test_results.box, 'map50', 'N/A')
                map = getattr(char_test_results.box, 'map', 'N/A')
                print(f"Test mAP50: {map50}")
                print(f"Test mAP50-95: {map}")
            except:
                print("Không thể truy cập test metrics của YOLOv8n")

        print("\n--- YOLOv8s Character Recognition ---")
        if char_val_results_v8s:
            try:
                map50 = getattr(char_val_results_v8s.box, 'map50', 'N/A')
                map = getattr(char_val_results_v8s.box, 'map', 'N/A')
                print(f"Validation mAP50: {map50}")
                print(f"Validation mAP50-95: {map}")
            except:
                print("Không thể truy cập metrics của YOLOv8s")

        if char_test_results_v8s:
            try:
                map50 = getattr(char_test_results_v8s.box, 'map50', 'N/A')
                map = getattr(char_test_results_v8s.box, 'map', 'N/A')
                print(f"Test mAP50: {map50}")
                print(f"Test mAP50-95: {map}")
            except:
                print("Không thể truy cập test metrics của YOLOv8s")

        print("\n--- Kết luận ---")
        print("YOLOv8s thường có độ chính xác cao hơn YOLOv8n nhưng chậm hơn và nặng hơn")
        print("YOLOv8n nhanh hơn và nhẹ hơn, phù hợp cho inference real-time")

    else:
        print("Không thể so sánh do một trong hai mô hình chưa được train thành công")

# Chạy so sánh
compare_models_performance()

In [ ]:
def extract_license_plate_text(image_path, plate_model, char_model):
    """
    Hàm tổng hợp để phát hiện biển số và nhận dạng ký tự chỉ bằng YOLO
    """
    try:
        # Đọc ảnh
        image = cv2.imread(image_path)
        if image is None:
            print(f"Không thể đọc ảnh: {image_path}")
            return None

        # Bước 1: Phát hiện biển số
        plate_results = plate_model(image)

        extracted_texts = []

        for result in plate_results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Lấy tọa độ bounding box
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

                    # Cắt vùng biển số
                    license_plate = image[y1:y2, x1:x2]

                    if license_plate.size > 0:
                        # Sử dụng mô hình YOLO nhận dạng ký tự
                        char_results = char_model(license_plate)
                        yolo_text = ""

                        # Sắp xếp các ký tự theo vị trí
                        char_detections = []
                        for char_result in char_results:
                            if char_result.boxes is not None:
                                for char_box in char_result.boxes:
                                    x_char = float(char_box.xyxy[0][0])  # Tọa độ x của ký tự
                                    class_id = int(char_box.cls[0])
                                    confidence = float(char_box.conf[0])

                                    # Lấy tên class từ model
                                    if hasattr(char_result, 'names') and class_id in char_result.names:
                                        char = char_result.names[class_id]
                                        char_detections.append((x_char, char, confidence))

                        # Sắp xếp theo tọa độ x (từ trái sang phải) và ghép thành chuỗi
                        char_detections.sort(key=lambda x: x[0])
                        yolo_text = ''.join([char for _, char, _ in char_detections])

                        # Tính confidence trung bình
                        avg_confidence = sum([conf for _, _, conf in char_detections]) / len(char_detections) if char_detections else 0.0

                        # Kết quả
                        result_dict = {
                            'bbox': (x1, y1, x2, y2),
                            'text': yolo_text,
                            'confidence': avg_confidence,
                            'char_details': char_detections,
                            'license_plate_image': license_plate
                        }

                        extracted_texts.append(result_dict)

        return extracted_texts

    except Exception as e:
        print(f"Lỗi trong quá trình xử lý: {e}")
        return None

In [ ]:
def test_yolo_ocr_system():
    """
    Test hệ thống OCR với cả hai mô hình YOLOv8n và YOLOv8s
    """
    if not plate_detection_model:
        print("Cần có mô hình phát hiện biển số để test!")
        return

    # Kiểm tra mô hình nào có sẵn
    models_to_test = []
    if character_model:
        models_to_test.append(("YOLOv8n", character_model))
    if character_model_v8s:
        models_to_test.append(("YOLOv8s", character_model_v8s))

    if not models_to_test:
        print("Cần có ít nhất một mô hình nhận dạng ký tự để test!")
        return

    # Lấy một số ảnh test từ dataset
    test_images_path = f"{dataset2_path}/test/images"

    if os.path.exists(test_images_path):
        test_images = os.listdir(test_images_path)[:5]  # Test 5 ảnh đầu tiên

        for img_name in test_images:
            img_path = os.path.join(test_images_path, img_name)
            print(f"\n{'='*60}")
            print(f"Xử lý ảnh: {img_name}")
            print('='*60)

            # Test với từng mô hình
            for model_name, char_model in models_to_test:
                print(f"\n--- Kết quả với {model_name} ---")
                results = extract_license_plate_text(img_path, plate_detection_model, char_model)

                if results:
                    for i, result in enumerate(results):
                        print(f"  Biển số {i+1}:")
                        print(f"    Text: {result['text']}")
                        print(f"    Confidence: {result['confidence']:.3f}")
                        print(f"    Chi tiết ký tự: {result['char_details']}")

                        # Hiển thị ảnh biển số đã cắt (chỉ lần đầu tiên)
                        if model_name == models_to_test[0][0]:
                            show_image(result['license_plate_image'], f"License Plate {i+1} from {img_name}")
                else:
                    print("    Không phát hiện được biển số")
    else:
        print(f"Không tìm thấy thư mục test: {test_images_path}")

# Chạy test
test_yolo_ocr_system()

In [ ]:
def save_models():
    """
    Lưu tất cả các mô hình đã huấn luyện
    """
    try:
        # Tạo thư mục lưu mô hình
        os.makedirs('/content/saved_models', exist_ok=True)

        # Lấy thư mục làm việc hiện tại
        current_dir = os.getcwd()
        print(f"Thư mục làm việc hiện tại: {current_dir}")

        if plate_detection_model:
            # Lưu mô hình phát hiện biển số
            plate_detection_model.save('/content/saved_models/plate_detection_model.pt')
            print("Đã lưu mô hình phát hiện biển số")

        if character_model:
            # Lưu mô hình nhận dạng ký tự YOLOv8n
            character_model.save('/content/saved_models/character_recognition_yolov8n.pt')
            print("Đã lưu mô hình nhận dạng ký tự YOLOv8n")

        if character_model_v8s:
            # Lưu mô hình nhận dạng ký tự YOLOv8s
            character_model_v8s.save('/content/saved_models/character_recognition_yolov8s.pt')
            print("Đã lưu mô hình nhận dạng ký tự YOLOv8s")

        # Copy best weights
        import shutil

        # Plate detection best weights
        plate_best_path = '/content/runs/detect/license_plate_detection/weights/best.pt'
        if os.path.exists(plate_best_path):
            shutil.copy(plate_best_path, '/content/saved_models/plate_detection_best.pt')
            print("Đã copy best weights cho phát hiện biển số")
        else:
            print(f"Không tìm thấy file: {plate_best_path}")

        # Character recognition YOLOv8n best weights
        char_v8n_best_path = '/content/runs/detect/character_recognition/weights/best.pt'
        if os.path.exists(char_v8n_best_path):
            shutil.copy(char_v8n_best_path, '/content/saved_models/character_recognition_yolov8n_best.pt')
            print("Đã copy best weights cho nhận dạng ký tự YOLOv8n")
        else:
            print(f"Không tìm thấy file: {char_v8n_best_path}")

        # Character recognition YOLOv8s best weights
        char_v8s_best_path = '/content/runs/detect/character_recognition_v8s/weights/best.pt'
        if os.path.exists(char_v8s_best_path):
            shutil.copy(char_v8s_best_path, '/content/saved_models/character_recognition_yolov8s_best.pt')
            print("Đã copy best weights cho nhận dạng ký tự YOLOv8s")
        else:
            print(f"Không tìm thấy file: {char_v8s_best_path}")

        print("\nCác file mô hình đã được lưu tại: /content/saved_models/")
        print("Danh sách file:")

        # Liệt kê các file đã lưu
        saved_files = os.listdir('/content/saved_models/')
        for file in saved_files:
            file_path = os.path.join('/content/saved_models/', file)
            file_size = os.path.getsize(file_path)
            print(f"  - {file} ({file_size/1024/1024:.2f} MB)")

    except Exception as e:
        print(f"Lỗi khi lưu mô hình: {e}")

        # Debug: Kiểm tra các đường dẫn
        paths_to_check = [
            '/content/runs/detect/license_plate_detection/weights/best.pt',
            '/content/runs/detect/character_recognition/weights/best.pt',
            '/content/runs/detect/character_recognition_v8s/weights/best.pt'
        ]

        print("\nKiểm tra các đường dẫn:")
        for path in paths_to_check:
            exists = os.path.exists(path)
            print(f"  {path}: {'✓' if exists else '✗'}")

# Lưu mô hình
save_models()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/saved_models /content/drive/MyDrive/

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

# Load models
plate_model = YOLO("/kaggle/working/saved_models/plate_detection_best.pt")
char_model = YOLO("/kaggle/working/saved_models/character_recognition_yolov8s_best.pt")

class CustomLicensePlateRecognizer:
    def __init__(self, plate_model, char_model):
        self.plate_model = plate_model
        self.char_model = char_model

    def recognize_from_path(self, image_path):
        image = cv2.imread(image_path)
        if image is None:
            print("❌ Không thể đọc ảnh.")
            return None
        return self.recognize_from_array(image)

    def recognize_from_array(self, image):
        results = []
        plate_results = self.plate_model(image)
        for result in plate_results:
            if result.boxes is not None:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    plate_img = image[y1:y2, x1:x2]
                    char_results = self.char_model(plate_img)
                    char_detections = []
                    for char_result in char_results:
                        if char_result.boxes is not None:
                            for char_box in char_result.boxes:
                                x_char = float(char_box.xyxy[0][0])
                                class_id = int(char_box.cls[0])
                                conf = float(char_box.conf[0])
                                char = char_result.names[class_id]
                                char_detections.append((x_char, char, conf))
                    char_detections.sort(key=lambda x: x[0])
                    text = ''.join([c for _, c, _ in char_detections])
                    avg_conf = sum([conf for _, _, conf in char_detections]) / len(char_detections) if char_detections else 0
                    results.append({'text': text, 'confidence': avg_conf, 'bbox': (x1, y1, x2, y2)})
        return results

    def visualize_results(self, image_path, results):
        image = cv2.imread(image_path)
        for result in results:
            x1, y1, x2, y2 = result['bbox']
            text = result['text']
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("Kết quả nhận dạng biển số")
        plt.show()

In [ ]:
# Khởi tạo recognizer
recognizer = CustomLicensePlateRecognizer(plate_model, char_model)

# Test ảnh
image_path = "/kaggle/input/test12/test.jpg"
print(f"🖼️ Đang test với ảnh: {image_path}")
results = recognizer.recognize_from_path(image_path)

if results:
    for i, res in enumerate(results):
        print(f"\n📋 Biển số {i+1}:")
        print(f"📝 Text: {res['text']}")
        print(f"🎯 Confidence: {res['confidence']:.2f}")
    recognizer.visualize_results(image_path, results)
else:
    print("❌ Không nhận dạng được biển số nào.")

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
import json
from datetime import datetime
import pandas as pd
import sys

class VideoLicensePlateRecognizer:
    def __init__(self, plate_model_path, char_model_path, output_dir="./output"):
        """
        Khởi tạo recognizer cho video

        Args:
            plate_model_path: Đường dẫn đến mô hình phát hiện biển số
            char_model_path: Đường dẫn đến mô hình nhận dạng ký tự
            output_dir: Thư mục lưu kết quả
        """
        try:
            self.plate_model = YOLO(plate_model_path)
            self.char_model = YOLO(char_model_path)
        except Exception as e:
            raise ValueError(f"Không thể tải model: {e}")

        self.output_dir = output_dir
        self.detected_plates = []  # Lưu trữ các biển số đã phát hiện

        # Tạo thư mục output
        os.makedirs(output_dir, exist_ok=True)

        # Kiểm tra môi trường GUI
        self.has_gui = self._check_gui_support()

    def _check_gui_support(self):
        """Kiểm tra xem môi trường có hỗ trợ GUI không"""
        try:
            # Thử tạo một window test
            test_img = np.zeros((100, 100, 3), dtype=np.uint8)
            cv2.imshow('test', test_img)
            cv2.waitKey(1)
            cv2.destroyWindow('test')
            return True
        except:
            return False

    def recognize_plate_in_frame(self, frame):
        """
        Nhận dạng biển số trong một frame

        Args:
            frame: Frame ảnh từ video

        Returns:
            List các biển số được phát hiện
        """
        results = []

        try:
            # Phát hiện biển số với verbose=False để giảm log
            plate_results = self.plate_model(frame, conf=0.5, verbose=False)

            for result in plate_results:
                if result.boxes is not None:
                    for box in result.boxes:
                        # Lấy tọa độ biển số
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                        plate_conf = float(box.conf[0])

                        # Kiểm tra tọa độ hợp lệ
                        if x2 <= x1 or y2 <= y1:
                            continue

                        # Cắt vùng biển số với kiểm tra bounds
                        h, w = frame.shape[:2]
                        x1, y1 = max(0, x1), max(0, y1)
                        x2, y2 = min(w, x2), min(h, y2)

                        plate_img = frame[y1:y2, x1:x2]

                        if plate_img.size == 0:
                            continue

                        # Nhận dạng ký tự trong biển số với verbose=False
                        char_results = self.char_model(plate_img, conf=0.3, verbose=False)
                        char_detections = []

                        for char_result in char_results:
                            if char_result.boxes is not None:
                                for char_box in char_result.boxes:
                                    x_char = float(char_box.xyxy[0][0])
                                    class_id = int(char_box.cls[0])
                                    conf = float(char_box.conf[0])
                                    char = char_result.names[class_id]
                                    char_detections.append((x_char, char, conf))

                        # Sắp xếp ký tự theo thứ tự từ trái sang phải
                        char_detections.sort(key=lambda x: x[0])

                        # Ghép thành chuỗi biển số
                        text = ''.join([c for _, c, _ in char_detections])
                        avg_conf = sum([conf for _, _, conf in char_detections]) / len(char_detections) if char_detections else 0

                        # Chỉ lấy biển số có độ tin cậy cao và có ít nhất 6 ký tự
                        if avg_conf > 0.5 and len(text) >= 6:
                            results.append({
                                'text': text,
                                'confidence': avg_conf,
                                'plate_conf': plate_conf,
                                'bbox': (x1, y1, x2, y2),
                                'plate_img': plate_img
                            })

        except Exception as e:
            print(f"⚠️ Lỗi khi nhận dạng frame: {e}")

        return results

    def draw_results_on_frame(self, frame, results, frame_number=0):
        """
        Vẽ kết quả lên frame

        Args:
            frame: Frame gốc
            results: Kết quả nhận dạng
            frame_number: Số thứ tự frame

        Returns:
            Frame đã được vẽ kết quả
        """
        annotated_frame = frame.copy()

        for result in results:
            x1, y1, x2, y2 = result['bbox']
            text = result['text']
            confidence = result['confidence']

            # Vẽ khung biển số
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

            # Tạo background cho text
            label = f"{text} ({confidence:.2f})"
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)

            # Đảm bảo text không vượt ra ngoài frame
            text_y = max(y1 - 5, text_height + 5)
            cv2.rectangle(annotated_frame, (x1, text_y - text_height - 5),
                         (x1 + text_width, text_y + 5), (0, 255, 0), -1)

            # Vẽ text
            cv2.putText(annotated_frame, label, (x1, text_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

        # Thêm timestamp
        timestamp = f"Frame: {frame_number}"
        cv2.putText(annotated_frame, timestamp, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        return annotated_frame

    def process_video(self, input_video_path, output_video_path=None, save_plates=True,
                     skip_frames=1, max_frames=None, show_progress=True):
        """
        Xử lý video để nhận dạng biển số

        Args:
            input_video_path: Đường dẫn video đầu vào
            output_video_path: Đường dẫn video đầu ra (nếu None sẽ tự tạo)
            save_plates: Có lưu ảnh biển số hay không
            skip_frames: Số frame bỏ qua để tăng tốc độ xử lý
            max_frames: Số frame tối đa để xử lý (None = xử lý hết)
            show_progress: Hiển thị tiến trình

        Returns:
            Dictionary chứa thống kê kết quả
        """
        # Mở video đầu vào
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            raise ValueError(f"Không thể mở video: {input_video_path}")

        # Lấy thông tin video
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if show_progress:
            print(f"📹 Thông tin video:")
            print(f"   - FPS: {fps}")
            print(f"   - Kích thước: {width}x{height}")
            print(f"   - Tổng frames: {total_frames}")
            print(f"   - GUI Support: {'Có' if self.has_gui else 'Không'}")

        # Tạo tên file output nếu không được cung cấp
        if output_video_path is None:
            base_name = os.path.splitext(os.path.basename(input_video_path))[0]
            output_video_path = os.path.join(self.output_dir, f"{base_name}_processed.mp4")

        # Khởi tạo video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

        if not out.isOpened():
            raise ValueError(f"Không thể tạo video output: {output_video_path}")

        # Khởi tạo biến thống kê
        frame_count = 0
        processed_frames = 0
        unique_plates = set()
        all_detections = []

        # Tạo thư mục lưu ảnh biển số
        if save_plates:
            plates_dir = os.path.join(self.output_dir, "detected_plates")
            os.makedirs(plates_dir, exist_ok=True)

        if show_progress:
            print(f"🔄 Bắt đầu xử lý video...")

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1

                # Bỏ qua frame nếu cần
                if frame_count % (skip_frames + 1) != 0:
                    out.write(frame)
                    continue

                # Giới hạn số frame xử lý
                if max_frames and processed_frames >= max_frames:
                    out.write(frame)
                    continue

                # Nhận dạng biển số
                results = self.recognize_plate_in_frame(frame)

                # Vẽ kết quả lên frame
                annotated_frame = self.draw_results_on_frame(frame, results, frame_count)

                # Lưu kết quả
                for i, result in enumerate(results):
                    plate_text = result['text']
                    timestamp = frame_count / fps if fps > 0 else 0

                    # Thêm vào danh sách phát hiện
                    detection_info = {
                        'frame': frame_count,
                        'timestamp': timestamp,
                        'plate_text': plate_text,
                        'confidence': result['confidence'],
                        'plate_confidence': result['plate_conf'],
                        'bbox': result['bbox']
                    }
                    all_detections.append(detection_info)

                    # Thêm vào set biển số unique
                    if result['confidence'] > 0.7:  # Chỉ lấy biển số có độ tin cậy cao
                        unique_plates.add(plate_text)

                    # Lưu ảnh biển số
                    if save_plates and result['confidence'] > 0.6:
                        try:
                            plate_filename = f"plate_{frame_count}_{i}_{plate_text}.jpg"
                            # Loại bỏ ký tự đặc biệt trong tên file
                            plate_filename = "".join(c for c in plate_filename if c.isalnum() or c in "._-")
                            plate_path = os.path.join(plates_dir, plate_filename)
                            cv2.imwrite(plate_path, result['plate_img'])
                        except Exception as e:
                            print(f"⚠️ Không thể lưu ảnh biển số: {e}")

                # Ghi frame đã xử lý
                out.write(annotated_frame)
                processed_frames += 1

                # Hiển thị tiến trình
                if show_progress and frame_count % 100 == 0:
                    progress = (frame_count / total_frames) * 100
                    print(f"   Tiến trình: {progress:.1f}% ({frame_count}/{total_frames})")

        except KeyboardInterrupt:
            print("\n⚠️ Đã dừng xử lý theo yêu cầu người dùng")

        except Exception as e:
            print(f"❌ Lỗi trong quá trình xử lý: {e}")

        finally:
            # Giải phóng tài nguyên - FIX CHO LỖI GUI
            cap.release()
            out.release()

            # Chỉ gọi destroyAllWindows nếu có hỗ trợ GUI
            if self.has_gui:
                try:
                    cv2.destroyAllWindows()
                except:
                    pass  # Bỏ qua lỗi nếu không thể destroy windows

        # Lưu kết quả ra file
        self.save_detection_results(all_detections, unique_plates)

        # Thống kê kết quả
        stats = {
            'total_frames': total_frames,
            'processed_frames': processed_frames,
            'total_detections': len(all_detections),
            'unique_plates': len(unique_plates),
            'output_video': output_video_path,
            'unique_plates_list': list(unique_plates)
        }

        if show_progress:
            self.print_summary(stats)

        return stats

    def save_detection_results(self, all_detections, unique_plates):
        """
        Lưu kết quả nhận dạng ra file
        """
        try:
            # Lưu chi tiết tất cả các phát hiện
            detections_file = os.path.join(self.output_dir, "all_detections.json")
            with open(detections_file, 'w', encoding='utf-8') as f:
                json.dump(all_detections, f, ensure_ascii=False, indent=2)

            # Lưu danh sách biển số unique
            unique_plates_file = os.path.join(self.output_dir, "unique_plates.txt")
            with open(unique_plates_file, 'w', encoding='utf-8') as f:
                for plate in sorted(unique_plates):
                    f.write(f"{plate}\n")

            # Lưu thành CSV để dễ phân tích
            if all_detections:
                df = pd.DataFrame(all_detections)
                csv_file = os.path.join(self.output_dir, "detections.csv")
                df.to_csv(csv_file, index=False, encoding='utf-8')

        except Exception as e:
            print(f"⚠️ Lỗi khi lưu kết quả: {e}")

    def print_summary(self, stats):
        """
        In tóm tắt kết quả
        """
        print(f"\n{'='*50}")
        print(f"📊 TÓM TẮT KẾT QUẢ XỬ LÝ VIDEO")
        print(f"{'='*50}")
        print(f"🎬 Tổng số frames: {stats['total_frames']}")
        print(f"⚡ Frames đã xử lý: {stats['processed_frames']}")
        print(f"🔍 Tổng số lần phát hiện: {stats['total_detections']}")
        print(f"🚗 Số biển số unique: {stats['unique_plates']}")
        print(f"📁 Video đầu ra: {stats['output_video']}")

        if stats['unique_plates_list']:
            print(f"\n📋 DANH SÁCH BIỂN SỐ PHÁT HIỆN:")
            for i, plate in enumerate(sorted(stats['unique_plates_list']), 1):
                print(f"   {i:2d}. {plate}")

        print(f"\n📂 Kết quả đã được lưu tại: {self.output_dir}")
        print(f"{'='*50}")

# Hàm tiện ích để sử dụng
def process_video_with_license_plate_recognition(
    input_video_path,
    plate_model_path="/kaggle/working/saved_models/plate_detection_best.pt",
    char_model_path="/kaggle/working/saved_models/character_recognition_yolov8s_best.pt",
    output_dir="./video_output",
    skip_frames=2,  # Xử lý mỗi 3 frames để tăng tốc
    max_frames=None,
    show_progress=True
):
    """
    Hàm wrapper để xử lý video nhận dạng biển số

    Args:
        input_video_path: Đường dẫn video đầu vào
        plate_model_path: Đường dẫn mô hình phát hiện biển số
        char_model_path: Đường dẫn mô hình nhận dạng ký tự
        output_dir: Thư mục lưu kết quả
        skip_frames: Số frame bỏ qua (0 = xử lý tất cả)
        max_frames: Số frame tối đa xử lý (None = xử lý hết)
        show_progress: Hiển thị tiến trình

    Returns:
        Dictionary chứa thống kê kết quả
    """
    try:
        recognizer = VideoLicensePlateRecognizer(
            plate_model_path=plate_model_path,
            char_model_path=char_model_path,
            output_dir=output_dir
        )

        return recognizer.process_video(
            input_video_path=input_video_path,
            skip_frames=skip_frames,
            max_frames=max_frames,
            show_progress=show_progress
        )

    except Exception as e:
        print(f"❌ Lỗi khởi tạo recognizer: {e}")
        return None

# Ví dụ sử dụng
if __name__ == "__main__":
    # Xử lý video
    input_video = "/kaggle/input/test78/plate_video (1) (online-video-cutter.com).mp4"

    try:
        stats = process_video_with_license_plate_recognition(
            input_video_path=input_video,
            output_dir="./license_plate_output",
            skip_frames=1,  # Xử lý mỗi 2 frames
            max_frames=1000,  # Xử lý tối đa 1000 frames để test
            show_progress=True
        )

        if stats:
            print("✅ Xử lý video hoàn thành!")
        else:
            print("❌ Xử lý video thất bại!")

    except Exception as e:
        print(f"❌ Lỗi khi xử lý video: {e}")